In [1]:
# 加载模型
from ultralytics import YOLO
import cv2
model = YOLO('/home/yuchen/Fdisk/Projects/YahBoom/YahBoom/python/best.pt')

In [16]:
!pip install streamlit

Looking in indexes: https://mirror.sjtu.edu.cn/pypi/web/simple
     ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
     - -------------------------------------- 0.3/9.1 MB ? eta -:--:--
     ------ --------------------------------- 1.6/9.1 MB 5.2 MB/s eta 0:00:02
     ------ --------------------------------- 1.6/9.1 MB 5.2 MB/s eta 0:00:02
     --------- ------------------------------ 2.1/9.1 MB 3.0 MB/s eta 0:00:03
     ---------- ----------------------------- 2.4/9.1 MB 2.4 MB/s eta 0:00:03
     -------------- ------------------------- 3.4/9.1 MB 2.9 MB/s eta 0:00:02
     ------------------ --------------------- 4.2/9.1 MB 3.1 MB/s eta 0:00:02
     ------------------- -------------------- 4.5/9.1 MB 3.1 MB/s eta 0:00:02
     -------------------- ------------------- 4.7/9.1 MB 2.8 MB/s eta 0:00:02
     --------------------- ------------------ 5.0/9.1 MB 2.5 MB/s eta 0:00:02
     ----------------------- ---------------- 5.2/9.1 MB 2.4 MB/s eta 0:00:02
     ----------

In [2]:
import cv2
import time

last_time = time.time()

# 打开摄像头（摄像头设备号通常是0，如果有多个摄像头可以试试1, 2等）
cap = cv2.VideoCapture(0)

# 设置视频流的宽和高（可选）
cap.set(3, 640)  # 宽
cap.set(4, 480)  # 高

while True:
    # 捕获视频流中的每一帧
    ret, frame = cap.read()

    if not ret:
        print("无法从摄像头读取图像")
        break

    # 使用YOLO模型进行推理（检测物体）
    results = model(frame)  # 直接使用frame输入

    # 每秒输出一次识别结果
    current_time = time.time()
    if current_time - last_time >= 1:  # 每秒输出一次
        last_time = current_time
        print(f"Detected {len(results[0].boxes)} objects")  # 输出检测到的物体数量

    # 提取检测结果：boxes, labels, and scores
    boxes = results[0].boxes  # 获取检测框（boxes）
    confidences = boxes.conf  # 获取每个框的置信度
    labels = boxes.cls  # 获取每个框的类标签（类别ID）

    if labels.numel() > 0:  # numel() 返回张量的元素总数
        print(labels[0].item())  # 打印第一个标签，使用 .item() 转换为纯 Python 值

    # 获取类别名称
    class_names = results[0].names  # 获取所有类的名称

    # 绘制检测框和标签
    for box, conf, label in zip(boxes.xywh, confidences, labels):
        x1, y1, w, h = box[:4]  # 获取框的位置和大小
        x1, y1, w, h = int(x1 - w / 2), int(y1 - h / 2), int(w), int(h)  # 将xywh转换为xyxy

        label_name = class_names[int(label)]  # 获取标签名称

        # 绘制框和标签
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{label_name} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 显示结果
    cv2.imshow("YOLOv8 Object Detection", frame)

    # 按 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()


0: 384x512 1 good, 233.5ms
Speed: 5.9ms preprocess, 233.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 512)



Detected 1 objects
tensor(4.)


0: 384x512 (no detections), 223.1ms
Speed: 1.5ms preprocess, 223.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 512)


RuntimeError: Boolean value of Tensor with no values is ambiguous